In [1]:
import psycopg2
from psycopg2.extras import RealDictCursor
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import date
from datetime import timedelta
from sklearn.cluster import KMeans
import matplotlib.pyplot  as plt

In [17]:
try:
    conn = psycopg2.connect(host='10.123.184.193', dbname='edw', user='sys', password='eos40d')
    cur = conn.cursor(cursor_factory=RealDictCursor)
except Exception:
    print('Fail to connect DB')

In [18]:
###기준일부터 오늘까지 데이터 불러오기 df1
#sql1="truncate table sbd.sbds_search_cnt_week_jk;"
#sql2="insert into sbd.sbds_search_cnt_week_jk select keyword,start_date,end_date,year_month,scale,indate from sbd.sbds_search_cnt_week where start_date >='20170101';"
sql3 = 'select keyword,min(scale),max(scale), avg(scale),stddev(scale) from sbd.sbds_search_cnt_week group by keyword;'

#jk는 19년 11월 이후 데이터모음 
#cur.execute(sql1)
#cur.execute(sql2)
cur.execute(sql3)
result = cur.fetchall()
df1 = pd.DataFrame(result)

###기준일 데이터 불러오기 df2
sql = "select keyword, scale from sbd.sbds_search_cnt_week_before_20170612 where start_date='20170102';"
cur.execute(sql)
result = cur.fetchall()
df2 = pd.DataFrame(result)

print("현재 총 키워드 수" , len(df1),"기준일(17년 1월) 데이터 수" , len(df2))

현재 총 키워드 수 453355 기준일(17년 1월) 데이터 수 385110


In [19]:
k = 15
# create model and prediction
model = KMeans(n_clusters=k,algorithm='auto')
k_df = df1[['avg']]
print(model.fit(k_df))
predict = pd.DataFrame(model.predict(k_df))
predict.columns=['predict']
final_df = pd.DataFrame(np.hstack((df1, predict)))
print(len(final_df))
final_df.head(10)

KMeans(n_clusters=15)
453355


,0,1,2,3,4,5
0,한일스테인레스,0,914,325.194503171247,191.518752155887,0
1,닥스키즈트레이닝복,0,62,0.418604651162791,3.2039158638197645,0
2,폴로집업니트,0,1791,266.030303030303,364.211225209670,0
3,린가디건,0,74,8.73106060606061,13.3527606839276685,0
4,밥식혜,0,3763,249.591966173362,271.164353226497,0
5,토니치토크렌치,0,345,128.748414376321,80.6555642541167917,0
6,울산성남롯데시네마,153,11263,2914.17045454545,2004.864109187967,11
7,순천국가정원물빛축제,0,1175,34.2045454545455,136.093025590003,0
8,소형행거,6,347,108.094696969697,64.2243167461109053,0
9,얼바인팬츠,0,23,1.03409090909091,3.1264194840397219,0


In [20]:
final_df=final_df.rename(columns={0:'keyword',1:'min',2:'max',3:'avg',4:'scale_std',5:'cluster_id'})
final_df=final_df.sort_values(by=['cluster_id','scale_std'],ascending=False)
final_df= pd.merge(final_df,df2, on= 'keyword', how ='left')
final_df = final_df.fillna(value = {'scale':0})
final_df.head(10)
### 201701월 정량 스케일

,keyword,min,max,avg,scale_std,cluster_id,scale
0,약국마스크,0,14418238,37443.6405919662,664867.97608286,14,917.0
1,임시공휴일,0,11376376,41223.8942917548,547895.79121871,14,617.0
2,부산데이트폭력,0,8047490,44718.4128787879,520674.32617150,14,0.0
3,포켓몬go,0,9881749,47167.4143763214,480695.64809980,14,20811.0
4,별똥별,1007,6420892,51571.3382663848,457174.54859242,14,1646620.0
5,디아크,2556,6914745,50994.5151515152,432416.12474452,14,0.0
6,평창롱패딩,0,6297838,38511.553030303,422893.35282104,14,0.0
7,이지은,4645,6555531,36687.8484848485,402841.66484048,14,0.0
8,심은하,6751,6464333,43317.0643939394,398049.04547980,14,0.0
9,원할머니보쌈,13044,8035101,37432.5031712474,368530.50430045,14,19915.0


In [21]:
##기준일 scale 0으로 2그룹 분리하기
final_df=final_df.drop_duplicates(subset=['keyword'])
scale0_df = final_df[final_df['scale']==0]
scale1_df = final_df[final_df['scale']>0]
scale1_idx=[i for i in range(0,1000000) if i%5 in [0]]
scale1_idx=scale1_idx[:len(scale1_df)]
scale0_idx=[i for i in range(0,1000000) if i%5 in [1,2,3,4]]
scale0_idx=scale0_idx[:len(scale0_df)]
scale1_df.index=scale1_idx
scale0_df.index=scale0_idx
fin_df= pd.concat([scale1_df,scale0_df])
fin_df=fin_df.rename(columns ={'scale':'20170102_scale'})
fin_df=fin_df.sort_index()
print(fin_df.head(10))
print(len(fin_df), )

    keyword   min       max               avg        scale_std cluster_id  \
0     약국마스크     0  14418238  37443.6405919662  664867.97608286         14   
1   부산데이트폭력     0   8047490  44718.4128787879  520674.32617150         14   
2       디아크  2556   6914745  50994.5151515152  432416.12474452         14   
3     평창롱패딩     0   6297838   38511.553030303  422893.35282104         14   
4       이지은  4645   6555531  36687.8484848485  402841.66484048         14   
5     임시공휴일     0  11376376  41223.8942917548  547895.79121871         14   
6       심은하  6751   6464333  43317.0643939394  398049.04547980         14   
7  인천초등생살인범     0   4120417  46679.4772727273  361701.45151593         14   
8       최다빈   571   4458007  51524.6742424242  358953.75808174         14   
9       윤손하  1334   5387453  40117.4696969697  355203.40314493         14   

   20170102_scale  
0           917.0  
1             0.0  
2             0.0  
3             0.0  
4             0.0  
5           617.0  
6           

In [22]:
##조 이름 정하고 조별 scale 0개수 확인하기
#fin_df=fin_df.drop_duplicates(subset=['keyword'])
fin_df=fin_df.reset_index(drop=True)
#fin_df=fin_df.sort_index()
fin_df['group_id'] = fin_df.index//5 +1
##group_id별로 0아닌거 개수 확인하기  
group_id=pd.DataFrame()
fin_df2 =fin_df
fin_df2['tmp_num'] = fin_df2.apply(lambda x: 1 if (x['20170102_scale'] > 0) else 0,axis = 1 )
group_id=fin_df2.groupby(by='group_id')['tmp_num'].sum()
group_id.unique()

array([1, 5], dtype=int64)

In [24]:
fin_df.groupby('group_id').count()['keyword'].unique()

array([5], dtype=int64)

In [25]:
fin_df.head(10)

,keyword,min,max,avg,scale_std,cluster_id,20170102_scale,group_id,tmp_num
0,약국마스크,0,14418238,37443.6405919662,664867.97608286,14,917.0,1,1
1,부산데이트폭력,0,8047490,44718.4128787879,520674.32617150,14,0.0,1,0
2,디아크,2556,6914745,50994.5151515152,432416.12474452,14,0.0,1,0
3,평창롱패딩,0,6297838,38511.553030303,422893.35282104,14,0.0,1,0
4,이지은,4645,6555531,36687.8484848485,402841.66484048,14,0.0,1,0
5,임시공휴일,0,11376376,41223.8942917548,547895.79121871,14,617.0,2,1
6,심은하,6751,6464333,43317.0643939394,398049.04547980,14,0.0,2,0
7,인천초등생살인범,0,4120417,46679.4772727273,361701.45151593,14,0.0,2,0
8,최다빈,571,4458007,51524.6742424242,358953.75808174,14,0.0,2,0
9,윤손하,1334,5387453,40117.4696969697,355203.40314493,14,0.0,2,0


In [26]:
w_date = datetime.now().strftime("%Y%m%d")
fin_df.to_csv(f'./keyword_list/{w_date}_keyword_list.txt')

In [27]:
fin_df[fin_df['group_id']==20000]

,keyword,min,max,avg,scale_std,cluster_id,20170102_scale,group_id,tmp_num
99995,퀸,3447,540613,18505.0359408034,57973.54246530,6,5391.0,20000,1
99996,서랍행거,117,885,397.871212121212,183.264831759238,0,0.0,20000,0
99997,여자반바지정장,0,958,115.219873150106,183.257443331573,0,0.0,20000,0
99998,리모와토파즈70,0,747,216.255813953488,183.257425488973,0,0.0,20000,0
99999,캡틴아메리카피젯스피너,0,1869,34.061310782241,183.256888914766,0,0.0,20000,0


In [28]:
fin_df[fin_df['group_id']==30000]

,keyword,min,max,avg,scale_std,cluster_id,20170102_scale,group_id,tmp_num
149995,핸드폰방수팩,90,5317,1036.45454545455,983.133813840616,0,696.0,30000,1
149996,와플카페,10,595,128.287878787879,83.4967753119757299,0,0.0,30000,0
149997,쿠론페즈토트,0,823,28.969696969697,83.4910745799129354,0,0.0,30000,0
149998,롯데리아김치버거,0,836,71.0681818181818,83.4910521543329150,0,0.0,30000,0
149999,대만카스테라레시피,0,668,26.0454545454545,83.4900202261272827,0,0.0,30000,0


In [30]:
fin_df[fin_df['group_id']==40000]

,keyword,min,max,avg,scale_std,cluster_id,20170102_scale,group_id,tmp_num
199995,박서보,66,4952,549.68710359408,590.881214387066,0,398.0,40000,1
199996,파빌리온가격,0,245,20.7121212121212,36.2811225366246234,0,0.0,40000,0
199997,하남토이킹덤,0,235,23.6136363636364,36.2798895213967859,0,0.0,40000,0
199998,토마토달걀스프,0,589,6.23044397463002,36.2796308356859573,0,0.0,40000,0
199999,여자배기진,0,223,25.8245243128964,36.2762013876087096,0,0.0,40000,0


In [91]:

scale1_df.index=scale1_idx

ValueError: Length mismatch: Expected axis has 302341 elements, new values have 272051 elements

In [63]:
final_df.to_csv('tmp.txt')

In [60]:
final_df[final_df[3]==0]

,0,1,2,3
262514,한일전,86679.2407407407,615082.08011715,0
398315,로건,76392.6111111111,467467.30157263,0
382556,빙그레,86623.6666666667,464892.25896829,0
450423,우체국쇼핑몰,85768.8518518519,446942.16979585,0
192736,전미선,88789.2592592593,432765.55203980,0
...,...,...,...,...
275032,아디다스키즈나그랑티셔츠,0,0,0
83215,벨카이라비용후즈후피부과압구정,0,0,0
275011,세인트제임스후리스,0,0,0
275008,보덤스텐텀블러,0,0,0


,keyword,scale_mean,scale_std,cluster_id
380513,코로나19,6255360.98148148,7459090.7913,1
316938,공적마스크,1099359.27777778,3508553.2362,2
365290,코로나,2593785.7962963,3370684.8801,2
63926,부부의세계,984955.851851852,2255199.4281,2
283122,이태원클라쓰,997172.611111111,1960422.7261,2
...,...,...,...,...
275032,아디다스키즈나그랑티셔츠,0,0,0
83215,벨카이라비용후즈후피부과압구정,0,0,0
275011,세인트제임스후리스,0,0,0
275008,보덤스텐텀블러,0,0,0


In [ ]:
##평균으로 클러스터 나누고 클러스터 내에서 표준편차로 정렬해서 크롤링 보내기 